In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def csv_to_pandas(file_path):
    df = pd.read_csv(file_path)
    return df

In [3]:
dataset_specs_all = csv_to_pandas('zarr_info.csv')
dataset_specs_all.index = dataset_specs_all['collection_name']
ds_specs = dataset_specs_all[['chunks', 'shape_dict', 'chunk_size_mb', 'number_coordinate_chunks', 'number_of_spatial_chunks']]
ds_specs


,chunks,shape_dict,chunk_size_mb,number_coordinate_chunks,number_of_spatial_chunks
collection_name,,,,,
power_901_monthly_meteorology_utc.zarr,"{'time': 492, 'lat': 25, 'lon': 25}","{'time': 492, 'lat': 361, 'lon': 576}",2.346039,43,332.697600
cmip6-pds_GISS-E2-1-G_historical_tas,"{'time': 600, 'lat': 90, 'lon': 144}","{'time': 1980, 'lat': 90, 'lon': 144}",29.663086,6,1.000000
aws-noaa-oisst-feedstock_reference,"{'time': 1, 'zlev': 1, 'lat': 720, 'lon': 1440}","{'time': 15044, 'zlev': 1, 'lat': 720, 'lon': ...",1.977539,4,1.000000
600_1440_29_CMIP6_daily_GISS-E2-1-G_tas.zarr,"{'time': 29, 'lat': 600, 'lon': 1440}","{'time': 730, 'lat': 600, 'lon': 1440}",95.581055,3,1.000000
600_1440_1_CMIP6_daily_GISS-E2-1-G_tas.zarr,"{'time': 1, 'lat': 600, 'lon': 1440}","{'time': 730, 'lat': 600, 'lon': 1440}",3.295898,3,1.000000
365_262_262_CMIP6_daily_GISS-E2-1-G_tas.zarr,"{'time': 365, 'lat': 262, 'lon': 262}","{'time': 730, 'lat': 600, 'lon': 1440}",95.577469,3,12.586679
cmip6-kerchunk,"{'time': 1, 'lat': 600, 'lon': 1440}","{'time': 730, 'lat': 600, 'lon': 1440}",3.295898,3,1.000000


In [4]:
for collection_name in ds_specs.index:
    file_path = f'results/{collection_name}_urls_stats.csv'
    if os.path.exists(file_path):
        results_df = csv_to_pandas(file_path)
        ds_specs.loc[collection_name, 'median response time (all tiles)'] = float(results_df[(results_df['Name'] == 'Aggregated')]['Median Response Time'])
        ds_specs.loc[collection_name, 'average response time (all tiles)'] = float(results_df[(results_df['Name'] == 'Aggregated')]['Average Response Time'])
        # skip the last row which is the "Aggregated"
        zooms = []
        for path in results_df['Name']:
            paths = path.split('/')
            if len(paths) > 2:
                zooms.append(paths[2])
            else:
                zooms.append('N/A')
        results_df['zoom'] = zooms
        zero_tile_results = results_df[results_df['zoom'] == '0']
        six_tile_results = results_df[results_df['zoom'] == '5']
        ds_specs.loc[collection_name, 'mean median response time - tile 0'] = np.mean(zero_tile_results['Median Response Time'])
        ds_specs.loc[collection_name, 'mean median response time - tile 5'] = np.mean(six_tile_results['Median Response Time'])        

ds_specs

/var/folders/jh/_03qbqf130l8hjh8rpc6f4_c0000gn/T/ipykernel_63205/445714263.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_specs.loc[collection_name, 'median response time (all tiles)'] = float(results_df[(results_df['Name'] == 'Aggregated')]['Median Response Time'])
/var/folders/jh/_03qbqf130l8hjh8rpc6f4_c0000gn/T/ipykernel_63205/445714263.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_specs.loc[collection_name, 'average response time (all tiles)'] = float(results_df[(results_df['Name'] == '

,chunks,shape_dict,chunk_size_mb,number_coordinate_chunks,number_of_spatial_chunks,median response time (all tiles),average response time (all tiles),mean median response time - tile 0,mean median response time - tile 5
collection_name,,,,,,,,,
power_901_monthly_meteorology_utc.zarr,"{'time': 492, 'lat': 25, 'lon': 25}","{'time': 492, 'lat': 361, 'lon': 576}",2.346039,43,332.697600,1300.0,1372.064749,8906.393573,1297.623254
cmip6-pds_GISS-E2-1-G_historical_tas,"{'time': 600, 'lat': 90, 'lon': 144}","{'time': 1980, 'lat': 90, 'lon': 144}",29.663086,6,1.000000,1400.0,1439.232433,4686.419815,1388.124432
aws-noaa-oisst-feedstock_reference,"{'time': 1, 'zlev': 1, 'lat': 720, 'lon': 1440}","{'time': 15044, 'zlev': 1, 'lat': 720, 'lon': ...",1.977539,4,1.000000,1200.0,1267.670636,6323.113333,1205.000714
600_1440_29_CMIP6_daily_GISS-E2-1-G_tas.zarr,"{'time': 29, 'lat': 600, 'lon': 1440}","{'time': 730, 'lat': 600, 'lon': 1440}",95.581055,3,1.000000,1300.0,1251.565044,4192.296207,1164.283061
600_1440_1_CMIP6_daily_GISS-E2-1-G_tas.zarr,"{'time': 1, 'lat': 600, 'lon': 1440}","{'time': 730, 'lat': 600, 'lon': 1440}",3.295898,3,1.000000,990.0,1052.238627,3795.709121,985.273919
365_262_262_CMIP6_daily_GISS-E2-1-G_tas.zarr,"{'time': 365, 'lat': 262, 'lon': 262}","{'time': 730, 'lat': 600, 'lon': 1440}",95.577469,3,12.586679,1200.0,1305.354084,6235.791065,1260.638450
cmip6-kerchunk,"{'time': 1, 'lat': 600, 'lon': 1440}","{'time': 730, 'lat': 600, 'lon': 1440}",3.295898,3,1.000000,1200.0,1205.197816,4552.947552,1171.255878
